In [21]:
%cd grade_prediction/
%pwd

[Errno 2] No such file or directory: 'grade_prediction/'
/home/yuhao/grade_prediction


'/home/yuhao/grade_prediction'

In [22]:
import numpy as np
import pandas as pd

# Initialise Model Manager

In [23]:
from ml.dataset import load_student_grade_dataset
from ml.model import PytorchModelManager

dataset = load_student_grade_dataset()
model_manager = PytorchModelManager(dataset)

model_manager.train(epoch=80)

Epoch: 0, loss=0.590, train_accuracy=0.377, test_accuracy=0.215
Epoch: 1, loss=0.585, train_accuracy=0.297, test_accuracy=0.190
Epoch: 2, loss=0.562, train_accuracy=0.392, test_accuracy=0.329
Epoch: 3, loss=0.576, train_accuracy=0.468, test_accuracy=0.354
Epoch: 4, loss=0.564, train_accuracy=0.434, test_accuracy=0.354
Epoch: 5, loss=0.569, train_accuracy=0.478, test_accuracy=0.392
Epoch: 6, loss=0.546, train_accuracy=0.487, test_accuracy=0.443
Epoch: 7, loss=0.523, train_accuracy=0.484, test_accuracy=0.405
Epoch: 8, loss=0.507, train_accuracy=0.544, test_accuracy=0.405
Epoch: 9, loss=0.514, train_accuracy=0.554, test_accuracy=0.367
Epoch: 10, loss=0.500, train_accuracy=0.538, test_accuracy=0.392
Epoch: 11, loss=0.485, train_accuracy=0.491, test_accuracy=0.418
Epoch: 12, loss=0.444, train_accuracy=0.604, test_accuracy=0.418
Epoch: 13, loss=0.483, train_accuracy=0.576, test_accuracy=0.443
Epoch: 14, loss=0.473, train_accuracy=0.617, test_accuracy=0.443
Epoch: 15, loss=0.485, train_accura

In [24]:
dataset.features

['school',
 'sex',
 'age',
 'address',
 'famsize',
 'Pstatus',
 'Medu',
 'Fedu',
 'Mjob',
 'Fjob',
 'reason',
 'guardian',
 'traveltime',
 'studytime',
 'failures',
 'schoolsup',
 'famsup',
 'paid',
 'activities',
 'nursery',
 'higher',
 'internet',
 'romantic',
 'famrel',
 'freetime',
 'goout',
 'Dalc',
 'Walc',
 'health',
 'absences',
 'G1',
 'G2']

In [52]:
data_df = dataset.data
features = dataset.features
res = []

for feature in features:
    if feature in dataset.numerical_features:
        feat_min = data_df[feature].min()
        feat_max = data_df[feature].max()

        feat_bins = 4
        labels = list(range(feat_min, feat_max + 1, int((feat_max + 1 - feat_min) / feat_bins)))
        if feat_max not in labels:
            labels.append(feat_max)
        
        feat_obj = {
          'name': feature.upper(),
          'type': 'numerical',
          'min': feat_min,
          'max': feat_max,
          'classes': labels,
          'values': data_df[feature].value_counts(bins = labels).tolist()
        }

    elif feature in dataset.categorical_features:
        category_value_count = data_df[feature].value_counts()
        feat_obj = {
          'name': feature.upper(),
          'type': 'categorical',
          'classes': category_value_count.index.tolist(),
          'values': category_value_count.values.tolist()
        }
    res.append(feat_obj)

In [53]:
res

[{'name': 'SCHOOL',
  'type': 'categorical',
  'classes': ['GP', 'MS'],
  'values': [349, 46]},
 {'name': 'SEX',
  'type': 'categorical',
  'classes': ['F', 'M'],
  'values': [208, 187]},
 {'name': 'AGE',
  'type': 'numerical',
  'min': 15,
  'max': 22,
  'classes': [15, 17, 19, 21, 22],
  'values': [284, 106, 4, 1]},
 {'name': 'ADDRESS',
  'type': 'categorical',
  'classes': ['U', 'R'],
  'values': [307, 88]},
 {'name': 'FAMSIZE',
  'type': 'categorical',
  'classes': ['GT3', 'LE3'],
  'values': [281, 114]},
 {'name': 'PSTATUS',
  'type': 'categorical',
  'classes': ['T', 'A'],
  'values': [354, 41]},
 {'name': 'MEDU',
  'type': 'numerical',
  'min': 0,
  'max': 4,
  'classes': [0, 1, 2, 3, 4],
  'values': [131, 103, 99, 62]},
 {'name': 'FEDU',
  'type': 'numerical',
  'min': 0,
  'max': 4,
  'classes': [0, 1, 2, 3, 4],
  'values': [115, 100, 96, 84]},
 {'name': 'MJOB',
  'type': 'categorical',
  'classes': ['other', 'services', 'at_home', 'teacher', 'health'],
  'values': [141, 103, 

In [51]:
data_df = dataset.data
feature = 'age'
feat_min = data_df[feature].min()
feat_max = data_df[feature].max()

feat_bins = 4
labels = list(range(feat_min, feat_max + 1, int((feat_max + 1 - feat_min) / feat_bins)))
if feat_max not in labels:
    labels.append(feat_max)
data_df[feature].value_counts(bins = labels).tolist()

[284, 106, 4, 1]

In [48]:
list(range(feat_min, feat_max + 1, int((feat_max + 1 - feat_min) / feat_bins)))

[15, 17, 19, 21]

In [33]:
dataset._data['age'].describe()

count    395.000000
mean      16.696203
std        1.276043
min       15.000000
25%       16.000000
50%       17.000000
75%       18.000000
max       22.000000
Name: age, dtype: float64

# Initialise Engine

In [4]:
from ml.cf_engine.engine import CFEnginePytorch
engine = CFEnginePytorch(dataset, model_manager)

In [5]:
query_instance = dataset._data.iloc[1]
query_instance

school             GP
sex                 F
age                17
address             U
famsize           GT3
Pstatus             T
Medu                1
Fedu                1
Mjob          at_home
Fjob            other
reason         course
guardian       father
traveltime          1
studytime           2
failures            0
schoolsup          no
famsup            yes
paid               no
activities         no
nursery            no
higher            yes
internet          yes
romantic           no
famrel              5
freetime            3
goout               3
Dalc                1
Walc                1
health              3
absences            4
G1                  5
G2                  5
G3                  D
Name: 1, dtype: object

In [6]:
import json
setting = json.loads("{\"changeable_attr\":[\"school\",\"sex\",\"age\",\"address\",\"famsize\",\"Pstatus\",\"Medu\",\"Fedu\",\"Mjob\",\"Fjob\",\"reason\",\"guardian\",\"traveltime\",\"studytime\",\"failures\",\"schoolsup\",\"famsup\",\"paid\",\"activities\",\"nursery\",\"higher\",\"internet\",\"romantic\",\"famrel\",\"freetime\",\"goout\",\"Dalc\",\"Walc\",\"health\",\"absences\",\"G1\",\"G2\"],\"cf_range\":{\"school\":{\"name\":\"school\",\"categories\":[\"GP\",\"MS\"]},\"sex\":{\"name\":\"sex\",\"categories\":[\"F\",\"M\"]},\"age\":{\"name\":\"age\",\"extent\":[15,23],\"min\":15,\"max\":23},\"address\":{\"name\":\"address\",\"categories\":[\"U\",\"R\"]},\"famsize\":{\"name\":\"famsize\",\"categories\":[\"GT3\",\"LE3\"]},\"Pstatus\":{\"name\":\"Pstatus\",\"categories\":[\"A\",\"T\"]},\"Medu\":{\"name\":\"Medu\",\"extent\":[0,5],\"min\":0,\"max\":5},\"Fedu\":{\"name\":\"Fedu\",\"extent\":[0,5],\"min\":0,\"max\":5},\"Mjob\":{\"name\":\"Mjob\",\"categories\":[\"at_home\",\"health\",\"other\",\"services\",\"teacher\"]},\"Fjob\":{\"name\":\"Fjob\",\"categories\":[\"teacher\",\"other\",\"services\",\"health\",\"at_home\"]},\"reason\":{\"name\":\"reason\",\"categories\":[\"course\",\"other\",\"home\",\"reputation\"]},\"guardian\":{\"name\":\"guardian\",\"categories\":[\"mother\",\"father\",\"other\"]},\"traveltime\":{\"name\":\"traveltime\",\"extent\":[1,5],\"min\":1,\"max\":5},\"studytime\":{\"name\":\"studytime\",\"extent\":[1,5],\"min\":1,\"max\":5},\"failures\":{\"name\":\"failures\",\"extent\":[0,4],\"min\":0,\"max\":4},\"schoolsup\":{\"name\":\"schoolsup\",\"categories\":[\"yes\",\"no\"]},\"famsup\":{\"name\":\"famsup\",\"categories\":[\"no\",\"yes\"]},\"paid\":{\"name\":\"paid\",\"categories\":[\"no\",\"yes\"]},\"activities\":{\"name\":\"activities\",\"categories\":[\"no\",\"yes\"]},\"nursery\":{\"name\":\"nursery\",\"categories\":[\"yes\",\"no\"]},\"higher\":{\"name\":\"higher\",\"categories\":[\"yes\",\"no\"]},\"internet\":{\"name\":\"internet\",\"categories\":[\"no\",\"yes\"]},\"romantic\":{\"name\":\"romantic\",\"categories\":[\"no\",\"yes\"]},\"famrel\":{\"name\":\"famrel\",\"extent\":[1,6],\"min\":1,\"max\":6},\"freetime\":{\"name\":\"freetime\",\"extent\":[1,6],\"min\":1,\"max\":6},\"goout\":{\"name\":\"goout\",\"extent\":[1,6],\"min\":1,\"max\":6},\"Dalc\":{\"name\":\"Dalc\",\"extent\":[1,6],\"min\":1,\"max\":6},\"Walc\":{\"name\":\"Walc\",\"extent\":[1,6],\"min\":1,\"max\":6},\"health\":{\"name\":\"health\",\"extent\":[1,6],\"min\":1,\"max\":6},\"absences\":{\"name\":\"absences\",\"extent\":[0,76],\"min\":0,\"max\":76},\"G1\":{\"name\":\"G1\",\"extent\":[3,20],\"min\":3,\"max\":20},\"G2\":{\"name\":\"G2\",\"extent\":[0,20],\"min\":0,\"max\":20}},\"num\":12,\"k\":32}")
setting['desired_class'] = np.repeat([[0,0,0,1]], repeats=setting['num'], axis=0)
res = engine.generate_counterfactual_examples([query_instance], setting=setting).valid
res.values.tolist()

/home/yuhao/miniconda3/envs/project/lib/python3.7/site-packages/torch/autograd/__init__.py:132: UserWarning: grad and param do not obey the gradient layout contract. This is not an error, but may impair performance.
grad.sizes() = [12, 58], strides() = [58, 1]
param.sizes() = [12, 58], strides() = [1, 12] (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370141920/work/torch/csrc/autograd/functions/accumulate_grad.h:170.)
  allow_unreachable=True)  # allow_unreachable flag


[1/1]  Epoch-0, time cost: 7.500s, loss: 0.976, iterations: 1999, validation rate: 0.833


[['GP',
  'F',
  17.0,
  'U',
  'GT3',
  'T',
  1.0,
  1.0,
  'at_home',
  'other',
  'course',
  'father',
  1.0,
  2.0,
  0.0,
  'no',
  'yes',
  'no',
  'no',
  'no',
  'yes',
  'yes',
  'no',
  5.0,
  3.0,
  3.0,
  1.0,
  1.0,
  3.0,
  23.0,
  9.0,
  13.0,
  'B',
  'B'],
 ['GP',
  'F',
  17.0,
  'U',
  'GT3',
  'T',
  1.0,
  1.0,
  'at_home',
  'other',
  'course',
  'father',
  1.0,
  2.0,
  0.0,
  'no',
  'yes',
  'no',
  'no',
  'no',
  'yes',
  'yes',
  'no',
  5.0,
  3.0,
  3.0,
  1.0,
  1.0,
  3.0,
  20.0,
  14.0,
  6.0,
  'B',
  'B'],
 ['GP',
  'F',
  17.0,
  'U',
  'GT3',
  'T',
  1.0,
  1.0,
  'at_home',
  'other',
  'course',
  'father',
  1.0,
  2.0,
  0.0,
  'no',
  'yes',
  'no',
  'no',
  'no',
  'yes',
  'yes',
  'no',
  5.0,
  3.0,
  3.0,
  1.0,
  1.0,
  3.0,
  4.0,
  11.0,
  10.0,
  'B',
  'B'],
 ['GP',
  'F',
  17.0,
  'U',
  'GT3',
  'T',
  1.0,
  1.0,
  'at_home',
  'other',
  'course',
  'father',
  1.0,
  2.0,
  0.0,
  'no',
  'yes',
  'no',
  'no',
  'no',
  

In [7]:
res

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,G3_pred
0,GP,F,17.0,U,GT3,T,1.0,1.0,at_home,other,...,3.0,3.0,1.0,1.0,3.0,23.0,9.0,13.0,B,B
1,GP,F,17.0,U,GT3,T,1.0,1.0,at_home,other,...,3.0,3.0,1.0,1.0,3.0,20.0,14.0,6.0,B,B
3,GP,F,17.0,U,GT3,T,1.0,1.0,at_home,other,...,3.0,3.0,1.0,1.0,3.0,4.0,11.0,10.0,B,B
4,GP,F,17.0,U,GT3,T,1.0,1.0,at_home,other,...,3.0,3.0,1.0,1.0,3.0,9.0,11.0,11.0,B,B
5,GP,F,17.0,U,GT3,T,1.0,1.0,health,other,...,3.0,3.0,1.0,1.0,3.0,4.0,8.0,11.0,B,B
6,GP,F,18.0,U,GT3,T,1.0,1.0,health,other,...,3.0,3.0,1.0,1.0,3.0,4.0,6.0,14.0,B,B
7,GP,F,17.0,U,GT3,T,1.0,1.0,health,other,...,3.0,3.0,1.0,1.0,3.0,4.0,5.0,14.0,B,B
8,GP,F,17.0,U,GT3,T,1.0,1.0,health,other,...,3.0,3.0,1.0,1.0,3.0,4.0,12.0,6.0,B,B
9,GP,F,17.0,U,GT3,T,1.0,1.0,health,other,...,3.0,3.0,1.0,1.0,3.0,4.0,10.0,8.0,B,B
10,GP,F,17.0,U,GT3,T,1.0,1.0,health,other,...,3.0,3.0,1.0,1.0,3.0,0.0,8.0,9.0,B,B


In [11]:
pd.DataFrame([query_instance, res.iloc[0]])

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,G3_pred
1,GP,F,17.0,U,GT3,T,1.0,1.0,at_home,other,...,3.0,3.0,1.0,1.0,3.0,4.0,5.0,5.0,D,NaN
0,GP,F,17.0,U,GT3,T,1.0,1.0,at_home,other,...,3.0,3.0,1.0,1.0,3.0,23.0,9.0,13.0,B,B


In [26]:
x, x_dash = query_instance.to_list(), res.iloc[2].to_list()[:-1]

for i in range(len(x)):
  if x[i] != x_dash[i]:
    print(dataset.columns[i]+":", x[i], "->", x_dash[i])

G1: 5 -> 11.0
G2: 5 -> 10.0
G3: D -> B


In [17]:
dataset.columns

['school',
 'sex',
 'age',
 'address',
 'famsize',
 'Pstatus',
 'Medu',
 'Fedu',
 'Mjob',
 'Fjob',
 'reason',
 'guardian',
 'traveltime',
 'studytime',
 'failures',
 'schoolsup',
 'famsup',
 'paid',
 'activities',
 'nursery',
 'higher',
 'internet',
 'romantic',
 'famrel',
 'freetime',
 'goout',
 'Dalc',
 'Walc',
 'health',
 'absences',
 'G1',
 'G2',
 'G3']